**XGBoost Model**

In [1]:

import os
print(os.listdir("../input"))
os.cpu_count()


['trainpredict', 'testpredict']


4

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X = pd.read_csv("../input/trainpredict/train.csv")
X_1 = pd.read_csv("../input/testpredict/test.csv")

In [4]:
train = X
test = X_1
print(train.isnull().sum(axis=0))
print(test.isnull().sum(axis = 0))

ID                   0
datetime             0
siteid         1212515
offerid              0
category             0
merchant             0
countrycode          0
browserid       608327
devid          1820299
click                0
dtype: int64
ID                  0
datetime            0
siteid         370776
offerid             0
category            0
merchant            0
countrycode         0
browserid      221906
devid          704619
dtype: int64


**Data  Pre Processing**
1. Add features of week day and hour
2. Update hour as range. Here, 24 hour has been divided into 6 parts
3. Label encode all features
4. For the missing values keep a separate label as -1


In [5]:
train['datetime'] = pd.to_datetime(train['datetime'])
train['day'] = train['datetime'].dt.weekday
train['hour'] = train['datetime'].dt.hour

test['datetime'] = pd.to_datetime(test['datetime'])
test['day'] = test['datetime'].dt.weekday
test['hour'] = test['datetime'].dt.hour

train['hour1'] = train['hour']%4
test['hour1'] = test['hour']%4

train = train.drop(columns=['ID', 'datetime', 'hour',])
test = test.drop(columns=['datetime', 'hour', ])

train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,day,hour1
0,4709696.0,887235,17714,20301556,e,Firefox,NaN,0,5,1
1,5189467.0,178235,21407,9434818,b,Mozilla Firefox,Desktop,0,2,1
2,98480.0,518539,25085,2050923,a,Edge,NaN,0,2,0
3,8896401.0,390352,40339,72089744,c,Firefox,Mobile,0,1,2
4,5635120.0,472937,12052,39507200,d,Mozilla Firefox,Desktop,0,5,0


In [6]:
from sklearn.preprocessing import LabelEncoder
train = train.astype('object')
test = test.astype('object')
col_with_null = ['siteid', 'browserid','devid']
cols = ['offerid', 'category', 'merchant', 'countrycode']
for col in col_with_null:
    lbl = LabelEncoder()
    temp_train = train[col]
    temp_test = test[col]
    mask = train[col].isnull()
    mask1 = test[col].isnull()
    train[col] = train[col].astype(str)
    test[col] = test[col].astype(str)
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))
    train[col] = train[col].where(~mask, temp_train)
    test[col] = test[col].where(~mask1, temp_test)
    
for col in cols:
    lbl = LabelEncoder()
    lbl.fit(list(train[col].values) + list(test[col].values))
    train[col] = lbl.transform(list(train[col].values))
    test[col] = lbl.transform(list(test[col].values))
    

In [7]:
train = train.replace('nan', np.NAN)
test = test.replace('nan',np.NAN)
train['siteid'].fillna(-1, inplace = True)
test['siteid'].fillna(-1, inplace = True)
train['browserid'].fillna(-1, inplace = True)
test['browserid'].fillna(-1, inplace = True)
train['devid'].fillna(-1, inplace = True)
test['devid'].fillna(-1, inplace = True)
train.head()

,siteid,offerid,category,merchant,countrycode,browserid,devid,click,day,hour1
0,112743.0,784773,48,127,4,2.0,-1.0,0,5,1
1,127360.0,157563,59,65,1,8.0,0.0,0,2,1
2,269055.0,458279,69,15,0,1.0,-1.0,0,2,0
3,240075.0,345067,117,507,2,2.0,1.0,0,1,2
4,140991.0,417948,36,276,3,8.0,0.0,0,5,0


In [8]:
print(test.head())
print(train.head())

        ID    siteid  offerid  category  ...    browserid  devid  day  hour1
0  IDFDJVI  199198.0   715248        48  ...          8.0    0.0    6      1
1  IDNWkTQ   82416.0   247936        36  ...          1.0    2.0    6      3
2  ID9pRmM  102642.0   823274        91  ...          8.0   -1.0    5      2
3  IDHaQaj   22971.0   541204        33  ...          1.0    2.0    6      2
4  IDT2CrF  130703.0   463364       130  ...          7.0   -1.0    6      1

[5 rows x 10 columns]
     siteid  offerid  category  merchant  ...    devid  click  day  hour1
0  112743.0   784773        48       127  ...     -1.0      0    5      1
1  127360.0   157563        59        65  ...      0.0      0    2      1
2  269055.0   458279        69        15  ...     -1.0      0    2      0
3  240075.0   345067       117       507  ...      1.0      0    1      2
4  140991.0   417948        36       276  ...      0.0      0    5      0

[5 rows x 10 columns]


In [9]:
train = train.astype(int)
for col in test.columns[1:]: test[col] = test[col].astype(int)


In [10]:
from sklearn.model_selection import train_test_split

Y_t = train['click'].astype('float')
X_t = train.drop(columns=['click'])
X_train, X_test, y_train, y_test = train_test_split(X_t, Y_t, test_size=0.25, random_state=69)

In [11]:
import xgboost as xgb
num_round = 1000
param = {'objective': 'binary:logistic',  
         'tree_method': 'hist', 
         'max_depth': 10,
         'eta': 0.2,
         'eval_metric': 'auc',
         'early_stopping_rounds':50,
         'nthread': 4,
         'max_bin':512,
         #'grow_policy':'lossguide',
         'random_state':69,
         }
dtrain = xgb.DMatrix(X_train, label=y_train, missing = -1)
dtest = xgb.DMatrix(X_test, label=y_test, missing = -1)
gbm = xgb.train(param, dtrain, num_round, evals=[(dtest, 'test')])
predicted = gbm.predict(xgb.DMatrix(X_test, missing = -1))

[07:51:31] Tree method is selected to be 'hist', which uses a single updater grow_fast_histmaker.
[07:51:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 656 extra nodes, 0 pruned nodes, max_depth=10
[0]	test-auc:0.96321
[07:51:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 756 extra nodes, 0 pruned nodes, max_depth=10
[1]	test-auc:0.96338
[07:51:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 802 extra nodes, 0 pruned nodes, max_depth=10
[2]	test-auc:0.963517
[07:51:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 882 extra nodes, 0 pruned nodes, max_depth=10
[3]	test-auc:0.963579
[07:51:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 918 extra nodes, 0 pruned nodes, max_depth=10
[4]	test-auc:0.963643
[07:51:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 944 extra nodes, 0 pruned nodes, max_depth=10
[5]	test-auc:0.963661
[07:51:55] /workspace/

[54]	test-auc:0.968108
[07:54:44] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 618 extra nodes, 0 pruned nodes, max_depth=10
[55]	test-auc:0.968146
[07:54:47] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1152 extra nodes, 0 pruned nodes, max_depth=10
[56]	test-auc:0.968318
[07:54:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 732 extra nodes, 0 pruned nodes, max_depth=10
[57]	test-auc:0.96834
[07:54:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1460 extra nodes, 0 pruned nodes, max_depth=10
[58]	test-auc:0.968596
[07:54:55] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 792 extra nodes, 0 pruned nodes, max_depth=10
[59]	test-auc:0.968615
[07:54:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1424 extra nodes, 0 pruned nodes, max_depth=10
[60]	test-auc:0.968777
[07:55:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1428 ext

[07:57:14] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1020 extra nodes, 0 pruned nodes, max_depth=10
[110]	test-auc:0.971166
[07:57:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1338 extra nodes, 0 pruned nodes, max_depth=10
[111]	test-auc:0.971216
[07:57:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1114 extra nodes, 0 pruned nodes, max_depth=10
[112]	test-auc:0.971246
[07:57:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1154 extra nodes, 0 pruned nodes, max_depth=10
[113]	test-auc:0.971286
[07:57:24] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 806 extra nodes, 0 pruned nodes, max_depth=10
[114]	test-auc:0.971291
[07:57:26] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1196 extra nodes, 0 pruned nodes, max_depth=10
[115]	test-auc:0.971346
[07:57:29] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1272 extra nodes, 0 pr

[164]	test-auc:0.97231
[07:59:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1122 extra nodes, 0 pruned nodes, max_depth=10
[165]	test-auc:0.972339
[07:59:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1174 extra nodes, 0 pruned nodes, max_depth=10
[166]	test-auc:0.97236
[07:59:36] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 762 extra nodes, 0 pruned nodes, max_depth=10
[167]	test-auc:0.972375
[07:59:39] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1230 extra nodes, 0 pruned nodes, max_depth=10
[168]	test-auc:0.972399
[07:59:41] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 660 extra nodes, 0 pruned nodes, max_depth=10
[169]	test-auc:0.972406
[07:59:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 964 extra nodes, 0 pruned nodes, max_depth=10
[170]	test-auc:0.972425
[07:59:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 93

[219]	test-auc:0.973145
[08:01:42] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1434 extra nodes, 0 pruned nodes, max_depth=10
[220]	test-auc:0.973148
[08:01:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1230 extra nodes, 0 pruned nodes, max_depth=10
[221]	test-auc:0.973163
[08:01:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1542 extra nodes, 0 pruned nodes, max_depth=10
[222]	test-auc:0.973161
[08:01:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1108 extra nodes, 0 pruned nodes, max_depth=10
[223]	test-auc:0.973165
[08:01:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 750 extra nodes, 0 pruned nodes, max_depth=10
[224]	test-auc:0.973171
[08:01:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1228 extra nodes, 0 pruned nodes, max_depth=10
[225]	test-auc:0.973187
[08:01:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots

...

[769]	test-auc:0.973811
[08:26:07] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1076 extra nodes, 0 pruned nodes, max_depth=10
[770]	test-auc:0.97381
[08:26:10] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1210 extra nodes, 0 pruned nodes, max_depth=10
[771]	test-auc:0.973798
[08:26:13] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 684 extra nodes, 0 pruned nodes, max_depth=10
[772]	test-auc:0.973799
[08:26:16] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1138 extra nodes, 0 pruned nodes, max_depth=10
[773]	test-auc:0.973796
[08:26:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1254 extra nodes, 0 pruned nodes, max_depth=10
[774]	test-auc:0.973787
[08:26:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 598 extra nodes, 0 pruned nodes, max_depth=10
[775]	test-auc:0.973794
[08:26:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 

[824]	test-auc:0.973763
[08:28:43] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 974 extra nodes, 0 pruned nodes, max_depth=10
[825]	test-auc:0.973754
[08:28:45] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 748 extra nodes, 0 pruned nodes, max_depth=10
[826]	test-auc:0.973751
[08:28:48] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 642 extra nodes, 0 pruned nodes, max_depth=10
[827]	test-auc:0.973752
[08:28:50] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1090 extra nodes, 0 pruned nodes, max_depth=10
[828]	test-auc:0.973746
[08:28:53] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 916 extra nodes, 0 pruned nodes, max_depth=10
[829]	test-auc:0.97375
[08:28:56] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1006 extra nodes, 0 pruned nodes, max_depth=10
[830]	test-auc:0.973743
[08:28:58] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 11

[879]	test-auc:0.973667
[08:31:15] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 856 extra nodes, 0 pruned nodes, max_depth=10
[880]	test-auc:0.973665
[08:31:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 822 extra nodes, 0 pruned nodes, max_depth=10
[881]	test-auc:0.973669
[08:31:19] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1158 extra nodes, 0 pruned nodes, max_depth=10
[882]	test-auc:0.973667
[08:31:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1228 extra nodes, 0 pruned nodes, max_depth=10
[883]	test-auc:0.973663
[08:31:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1028 extra nodes, 0 pruned nodes, max_depth=10
[884]	test-auc:0.973663
[08:31:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1104 extra nodes, 0 pruned nodes, max_depth=10
[885]	test-auc:0.973666
[08:31:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots,

[934]	test-auc:0.973571
[08:33:46] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1068 extra nodes, 0 pruned nodes, max_depth=10
[935]	test-auc:0.973566
[08:33:49] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1142 extra nodes, 0 pruned nodes, max_depth=10
[936]	test-auc:0.973561
[08:33:52] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 918 extra nodes, 0 pruned nodes, max_depth=10
[937]	test-auc:0.973561
[08:33:54] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1350 extra nodes, 0 pruned nodes, max_depth=10
[938]	test-auc:0.973558
[08:33:57] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1282 extra nodes, 0 pruned nodes, max_depth=10
[939]	test-auc:0.973558
[08:34:01] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1176 extra nodes, 0 pruned nodes, max_depth=10
[940]	test-auc:0.973558
[08:34:04] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots

[989]	test-auc:0.973464
[08:36:17] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 978 extra nodes, 0 pruned nodes, max_depth=10
[990]	test-auc:0.973464
[08:36:20] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 908 extra nodes, 0 pruned nodes, max_depth=10
[991]	test-auc:0.973463
[08:36:22] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1050 extra nodes, 0 pruned nodes, max_depth=10
[992]	test-auc:0.973464
[08:36:25] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1178 extra nodes, 0 pruned nodes, max_depth=10
[993]	test-auc:0.973457
[08:36:28] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 980 extra nodes, 0 pruned nodes, max_depth=10
[994]	test-auc:0.973461
[08:36:31] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 1168 extra nodes, 0 pruned nodes, max_depth=10
[995]	test-auc:0.973458
[08:36:34] /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 

In [12]:
from sklearn.metrics import confusion_matrix, roc_auc_score, accuracy_score as auc 
predicted = gbm.predict(xgb.DMatrix(X_test, missing = -1))
print(confusion_matrix(y_test, np.round(predicted)))
print(roc_auc_score(y_test, np.round(predicted)))
print(auc(y_test, np.round(predicted)))


[[2906408   18832]
 [  41522   67691]]
0.8066847017209796
0.980110418582855


In [13]:

preds = gbm.predict(xgb.DMatrix(test[test.columns[1:]], missing = -1))
sub = pd.DataFrame({'ID':test['ID'], 'click':preds})
sub.to_csv('submission_xg.csv', index=False)